## Intro Notebook
### Rice Data Science - DEEP Team 3
#### [Insert Name Here]

This notebook is designed as an environment to get an intro to doing data wrangling with pandas as well as starting to get insights into the thyroid data set. To begin we need to load the necessary packages.

In [ ]:
# Loading in packages
import pandas as pd
import numpy as np

Next we want to import the thyroid csv file into the notebook. One way to do this in colab is to first download the data set on to your computer and then upload the file into colab by clicking the folder icon on the left hand side of screen under the key and search icons. Once you open this tab, directly under files there should be an upload icon that allows you to upload the file for use in the Colab notebook. Now we can load the csv file into our notebook using pd.read_csv

In [ ]:
# CODE HERE: Read thyroid data into notebook using pandas and assign it to a variable named `thyroid_dat`


### Part 1: First Steps When Working on New Data

In any data science project, there's a few things you will want to focus on once you have got your data together in a usable format:
1. How much data do you have?
2. What does your data look like?
3. What types of data do you have?
4. How much data is missing (and why is it missing)?
5. How is your data distributed?

Some of these are more open ended than others and some will require tools we will learn in later workshops, but we can start by answers a couple of them using simple methods.

Let's start with figuring out how much data we are working with; `based on what we learned from the previous workshop, what method/attribute should we use?`

In [ ]:
# CODE HERE: Print the number of rows and columns in `thyroid_dat`


I would consider this to be a moderate amount of data; not in the millions or billions of observations to be considered "big data" (where you have too much data to load into memory), but also not a few hundred rows where you need to be much more careful about the statistical validity of the methods you use. In terms of features, I would also say this is a moderate to high amount; it will be somewhat difficult to completely understand 31 different variables, but it is not on the scale of having hundreds or more features where you likely would need to find workarounds to comprehend all the data you are working with. Overall, I would say this is quite a manageble data set so far.

Next we would like to get some sense of what the data actually looks like; `what approaches were covered in the workshop to directly look at our data?`

In [ ]:
# CODE HERE: Take a look at `thyroid_dat` using pandas


We already took some time to look at this data set and state some things that were notable about it. One thing mentioned was that the data include multiple data types. Pandas makes it extremely easy to summarize what data types we have in a couple of ways. `How do we display the data types of every column of a data set?`

In [ ]:
# CODE HERE: Print the data types of every column in the `thyroid_dat` data frame


What you will notice is that we have a mix of 64 bit integers, 64 bit floats, and object data types in this data set. Note that an object data type can be thought of as a general data type, although in most cases it indicates character or string data. `Do you notice any differences between the columns with an object data type?`

To better understand the differences between these columns we can check how many unique entries there are in each object column. `What attribute/method would be useful to do this?`

In [ ]:
# CODE HERE: 1. Filter the data set to only include columns with an object data type 2. Check how many unique entries are in these columns


Next, we would like to take a look at missing data. You will notice when we used the `.head()` method there were some `NaN` values, and we would like to better understand how much missing data we are working with before going much further.

In [ ]:
# CODE HERE: Print out the number of missing rows per column (consider using `.isna()`)


Unfortunately, we have enough columns that the output gets truncated, so let's try again filtering for columns that have at least one missing value.

In [ ]:
# Number of missing rows by column
print(thyroid_dat.loc[:, thyroid_dat.isna().sum() > 1].isna().sum(), "\n")

# Proportion of missing rows to total rows
print(thyroid_dat.loc[:, thyroid_dat.isna().sum() > 1].isna().sum() / thyroid_dat.shape[0] * 100)

From this we can see that the `TBG` variable is missing for more than 96% of the data, and the `T3` variable is missing for more than a quarter of the observations. Now before we start trying to deal with this missing data we should first ask ourselves, `what reasons could there be for this data to be missing?`

1. Structural Missingness: Data is missing because it should be missing. \
**Example:** Data about types of meat consumed is missing for a herbivore.
2. Missing Completely at Random (MCAR): Data is missing for no reason. \
**Example:** You take a data set and replace 10% of the data with missing values.
3. Missing at Random (MAR): Data is more likely to be missing due to predictable relationship. \
**Example:** People are less likely to answer a question about using ChatGPT for homework if they are under the age of 22.
4. Missing not at Random (MNAR): Data is missing for unclear reasons. \
**Example:** Economic data about wheat production is missing for Sweden, Kazakhstan, Australia, and South Africa.

Note that structural missingness is a special case of MNAR, however the main difference is that while structural missingness is easy to explain and often to model, the vast majority of data sets with data that is MNAR can run into massive issues when drawing generalizable conclusions.

As you might be able to tell, figuring out why data is missing can be an analysis project on its own depending on the complexity of the data, and for now we will wait to dive deeper in our analysis until we have more tools to work with.

The final major thing we want to get a handle on is how our data is distributed. For now, we will stick to numerical summaries of the data, but you could also think of things like histograms and box plots as being useful tools for this. `What Pandas methods can you use to get an idea of the distribution of the data?`

In [ ]:
# CODE HERE: Get a numerical summary of each numerical variable in the `thyroid_dat` data frame


While without good domain knowledge it is hard to get much insight from a lot of these summarys, it is clear that the numerical columns have fairly different scales, which we may need to account for going forward. We can also notice that there is something odd with the `age` column as the maximum age is apparently 65526 which obviously does not make much sense. When we get around to cleaning the data this is something to keep in mind.

However we don't want to be limited to just understanding the distribution of the numerical variables, and thankfully we can fairly easily describe categorical columns by just finding the number of observations by class. `What Pandas method counts the number of observations by value?`

In [ ]:
# Find distribution of boolean variables
thyroid_dat.loc[:, thyroid_dat.nunique() == 2].apply(pd.Series.value_counts)

In [ ]:
# Find distribution of target variable
thyroid_dat["target"].value_counts()

Naturally you can do through each categorical variable and call `value_counts()` on it, although we are just aiming to get a general idea of how our data is structured, and we can take a closer look later on.

### Part 2: Data Cleaning

Once we have a basic grasp of our data through the process above we are ready to start doing data cleaning. Data cleaning comprises of steps taken to make a data set more usable whether this be handling missing data, fixing incorrect data, etc. Data cleaning and wrangling is almost always the majority of time you will spend when doing a data science project and as such it can get quite complicated depending on the data you are working with. Thankfully our data is quite straightforward and high quality, so we will only need to take a few steps to get it into good shape.

For starters, let's take a look at that weird issues we found with age, where the maximum age was 65,526. Let's look at all the rows where age is higher than 100 to see if we can spot unreasonable outliers.

In [ ]:
# CODE HERE: Print all the rows of `thyroid_dat` where age is greater than 100


From this we can see that there is 4 rows where a patient has an obviously incorrect age. There is a few ways we could handle this. First, we have enough data that we could probably get away with just dropping these columns. However, if we don't want to do this then instead we could change these ages to missing values and deal with them when we deal with missing values.

In [ ]:
# Replace any values of age that are greater than 100 with NaN
thyroid_dat['age'] = thyroid_dat['age'].apply(lambda x: np.NaN if x > 100 else x)

It seems as though all of our columns are the correct type (i.e. no numerical columns are encoded as strings), so we can move on to dealing with the missing data. There are many different ways to deal with missing values; in the kickoff meeting you all mentioned things like dropping rows with missing values, imputing missing values with the mean, etc. Let's go over the pros and cons of some of these methods.

#### 1. Dropping Rows with Missing Vlaues
**Pros:** Quick and easy to perform; don't need to worry about changing distribution of data. \
**Cons:** Easy to drop too much data; dropped rows may contain valuable information; tacitly assumes data is MCAR. \
**Use Cases:**
1. Data sets with relatively small amounts of missing data.
2. Data sets where some columns are missing data in most or all columns.
3. Data sets with rows missing the response/target variable.

**Notes:** \
Dropping rows is one of the most rudimentary ways to deal with missing values, and while it does have legitimate use cases you do need to be very careful and ensure that dropping all rows with even one missing value does not lead to you dropping 96% of your dataset like it would in our case. Even more important is to ensure that dropping rows, even if it is just a few, is not resulting in you losing out on information in the dataset.


In [ ]:
# CODE HERE: Drop every row in the data set that has a missing value (do NOT assign it to `thyroid_dat` and ensure inplace=False)



#### 2. Replacing Missing Values with the Mean (or Median or Mode)
**Pros:** Quick and easy to perform; let's you retain all the rows in your data set. \
**Cons:** Reduces variance of the variable; imputation may not make logical sense. \
**Use Cases:**
1. Data sets with relatively small amounts of missing data.
2. Very large data sets where other techniques are too computationally intensive.

**Notes:** \
Imputing a single value for all missing values in a column is also very rudimentary and can get you in a lot of trouble in some cases. Ultimately, the thing to keep in mind is that imputing a single value is not simply going to replace the missing values, it is also going to change the data itself and the relationships within it. This can result in whatever analysis or modeling you do on this data to be misleading and make wrong conclusions because the data you are working with is not representative anymore of reality.


In [ ]:
# CODE HERE: Impute the age column using the mean


print('Number of null age values: ', thyroid_dat.isna().sum()['age']) # Code to check how many entries in age are null


#### 3. Replacing Missing Values with a Randomly Sampled Value
**Pros:** Fairly quick and easy to perform; let's you retain all the rows in your data set; typically will not change distribution of data. \
**Cons:** Imputation may not make logical sense; effectively just adding noise to the data. \
**Use Cases:**
1. Data sets with relatively small amounts of missing data.
2. Very large data sets where other techniques are too computationally intensive.

**Notes:** \
Replacing each missing value in a column with a randomly picked value already within the column can be an quick imputation strategy that does not have the con of mean imputation where you are changing the distribution of the data. However, the fact that you are randomly picking the value means that it inherently holds no actual information that would be useful for analysis or modeling. This can reduce predictive performance and make trends and other relationships harder to spot in analysis.


In [ ]:
# Example of how to impute using a randomly sampled value
non_missing_values = thyroid_dat['sex'].dropna()
thyroid_dat['sex'] = thyroid_dat['sex'].apply(lambda x: np.random.choice(non_missing_values) if pd.isnull(x) else x)
print('Number of null sex values: ', thyroid_dat.isna().sum()['sex'])


#### 4. Replacing Missing Values using Machine Learning
**Pros:** In the best case can replace missing values with actually meaningful values. \
**Cons:** Much more time-intensive even with simple models; no guarantee that imputations are useful. \
**Use Cases:**
1. Data sets with medium to large amounts of missing data in some columns.
2. Data sets where there are clear relationships between variables and missingness.
3. Data sets where simple techniques produce poor results.
4. Data sets with correlated predictor variables.

**Notes:** \
Using a machine learning model to impute missing values is a much more accurate way to do imputation than methods discussed so far. It is typically done using non-parametric models like k-means or random forest. Even for a simple model like k-means, it will take much longer to perform compared to simple techniques mentioned above. This also means you will need to deal with another model which can increase the complexity of a project substantially. While I have mentioned quite a few drawbacks, I would say in most cases you should use a machine learning based approach as it should reduce potential headaches caused by the other methods that only show up at the end of the project.


In [ ]:
# Example of doing machine learning imputation using K Nearest Neighbors
# Note K-NN imputation only works for numeric data
from sklearn.impute import KNNImputer

# Initialize KNNImputer with a specified number of neighbors (default is 5)
imputer = KNNImputer(n_neighbors=3)

# Apply the KNN imputer to the DataFrame
df_imputed = pd.DataFrame(imputer.fit_transform(thyroid_dat.select_dtypes(include = ["int", "float"])), columns=thyroid_dat.select_dtypes(include = ["int", "float"]).columns)
df_imputed.isna().sum()


#### 5. Multiple Imputation
**Pros:** Can greatly reduce random effects of a single imputation on results. \
**Cons:** Multiplies the amount of computational work of all subsequent data pipeline steps; much harder to do data analysis on multiple data sets concurrently than one. \
**Use Cases:**
1. Technically can be applied to any of the previous methods, but is more useful for non-deterministic methods like random sampling and machine learning.
2. Especially useful for data sets missing a lot of data.
3. When you need to quantify the amount of uncertainty in missing data.

**Notes:** \
Multiple imputation can be an effective imputation strategy if it appears as though the results of your analysis/modeling are highly affected by imputed values. In my experience it is much more time consuming to be doing analysis on multiple slightly different data sets, and you will want to have good helper functions to reduce the amount of manual work you need to do when training models on each data set and combining results. One really useful application of multiple imputation is that it allows you to understand how much uncertainty there is in the missing data which can be critical when doing statistical inference.

We won't dive into how to do multiple imputation as it takes a fair bit of set up, but it is a good thing to think about when doing future data projects.

#### Numeric Encoding

Most models require all input data to be numeric, so we need to encode all of the boolean and string variables into a numeric format. This is fairly simple for boolean variables as we just need to set one class to 0 and the other to 1. It gets much more complicated for string variables especially free text, but we will not need to worry about that for this data set. What we do need to worry about is string variables that represents multiple classes. Before that let's convert all our boolean variables to 0's and 1's.

In [ ]:
# CODE HERE: Encode boolean variables as 0's and 1's (there is a single pandas function that allows you to do this)
# Note you need to apply the transformation on only the boolean columns


Next we want to deal with the categorical columns that have multiple classes within them rather than just two. To do this we first need to identify which columns these are.

In [ ]:
# CODE HERE: Which columns are categorical and have more than 2 categories?


We have two multi-class categorical columns: `referral_source` with 6 different classes, and `target` with 32 different classes. Given that there are relatively few classes in `referral_source` we can make use of **one-hot encoding** where we create 5 columns for 5 of the classes, and each column is an indicator of whether that row has that particular class. One-hot encoding is a more general case of the encoding we did with the boolean variables, so we can reuse the same pandas function for it.

In [ ]:
# CODE HERE: Encode referral_source using one-hot encoding


Now for the `target` variable we need to be much more careful as this is what we want to predict. To get a better idea about how to deal with this variable let's take a look at what the classes actually are.

In [ ]:
# CODE HERE: Print the unique values of target


What you will notice is that alot of these are combinations of multiple letters. Another important thing to notice is that some of these classes have a line between them. Take a look at the data set information document; `what does this tell us about this patient?`

### Part 3: Encoding Target Variable

This will be a more open-ended section as there is no go to approach for encoding the `target` variable properly. Think about how you can encode all the inherent information between the different classes while still retaining the fact that some combinations represent multiple conditions and others represent conditions that are not completely clear.

In [ ]:
# CODE HERE: Encode target variable as numbers